In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install wandb --upgrade
import wandb
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from PIL import Image

In [3]:
################################################################
# Preparing training (without augmentation) and validation set 
################################################################
# Preparing training and validation sets without augmentation
# Loading data from directory
data_dir = pathlib.Path('/content/drive/MyDrive/inaturalist_12K/train') # Set path to the right directory
train_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = data_dir,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.2,
                      subset = 'training')

val_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = data_dir,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.2,
                      subset = 'validation')

# Retaining 25 percent of train and validation data and discarding the rest
len_train, len_val = len(train_data), len(val_data)
train_data = train_data.take(int(0.25*len_train))
val_data = val_data.take(int(0.25*len_val))

Found 10000 files belonging to 10 classes.
Using 8000 files for training.
Found 10000 files belonging to 10 classes.
Using 2000 files for validation.


In [4]:
train_data

<TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [5]:
################################################################
# Preparing training set with augmentation 
################################################################
train_data_augmenter = ImageDataGenerator(
                            rescale = None,
                            rotation_range = 20,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            brightness_range = [0.2, 1.5],
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            horizontal_flip=True,
                            data_format = 'channels_last',
                            validation_split = 0.2)        #Specifying parameters for augmentation of training data

val_data_augmenter = ImageDataGenerator(validation_split = 0.2) #No augmentation of validation data

train_aug_gen = train_data_augmenter.flow_from_directory(data_dir, shuffle = True, \
                                                         seed = 17, subset = 'training')
val_aug_gen = val_data_augmenter.flow_from_directory(data_dir, shuffle = True, \
                                                     seed = 17, subset = 'validation')

train_aug_data = tf.data.Dataset.from_generator(
                    lambda: train_aug_gen,
                    output_types = (tf.float32, tf.float32),
                    output_shapes = ([None, 256, 256, 3], [None, 10]))

val_aug_data = tf.data.Dataset.from_generator(
                  lambda: val_aug_gen,
                  output_types = (tf.float32, tf.float32),
                  output_shapes = ([None, 256, 256, 3], [None, 10]))

# Retaining 25 percent of train and validation data and discarding the rest
train_aug_data = train_aug_data.take(int(0.25*len_train))
val_aug_data = val_aug_data.take(int(0.25*len_val))

Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [6]:
train_aug_data

<TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [7]:
###############################################
# Listing the hyperparameters in wandb config 
###############################################
sweep_config = {'name': 'Bayesian Sweep', 'method': 'bayes'}
sweep_config['metric'] = {'name': 'val_acc', 'goal': 'maximize'}
parameters_dict = {
                   'first_layer_filters': {'values': [32, 64]},
                   'filter_org': {'values': [0.5, 1, 2]}, # Halving, same, doubling in subsequent layers
                   'data_aug': {'values': [False, True]},
                   'batch_norm': {'values': [False, True]}, 
                   'dropout': {'values': [0.0, 0.2, 0.3]},
                   'kernel_size': {'values': [3, 5, 7]},
                   'dense_size': {'values': [64,512,1024]},
                   'activation': {'values': ['relu']},
                   'num_epochs': {'values': [10,15,20]}, 
                   'optimizer': {'values': ['adam']},
                   'conv_layers': {'values': [5]}
                  }
sweep_config['parameters'] = parameters_dict


In [8]:
#####################################
# Defining the train function
#####################################
def CNN_train(config=sweep_config):
    with wandb.init(config=config):
        config = wandb.init().config
        wandb.run.name = 'firstLayerFilters_{}_filterOrg_{}_dataAug_{}_batchNorm_{}_dropout_{}_kerSize_{}_denseSize_{}'\
                         .format(config.first_layer_filters, config.filter_org, config.data_aug, config.batch_norm, config.dropout, config.kernel_size, config.dense_size)
        
        ###########################################
        # Initializing the model architecture
        ###########################################
        inputs = tf.keras.Input(shape = (256, 256, 3))
        x = Rescaling(scale = 1.0/255)(inputs)
        filter_sizes = [int(config.first_layer_filters*(config.filter_org**layer_num)) for layer_num in range(config.conv_layers)]
        ker_size = config.kernel_size

        # Apply some convolution and pooling layers
        for layer_num in range(config.conv_layers):
            x = layers.Conv2D(filters = filter_sizes[layer_num], kernel_size = (ker_size, ker_size))(x)
            if config.batch_norm:
                x = layers.BatchNormalization(axis = -1)(x)
            x = layers.Activation(config.activation)(x)
            x = layers.MaxPooling2D(pool_size = (2, 2))(x)            
                
        # Dense Layer
        x = layers.Flatten()(x)
        x = layers.Dense(config.dense_size)(x)
        if config.batch_norm:
            x = layers.BatchNormalization(axis = -1)(x)
        x = layers.Activation(config.activation)(x)
        if config.dropout > 0:
            x = layers.Dropout(rate = config.dropout)(x)        

        # Output Layer
        outputs = layers.Dense(10, activation ='softmax')(x)
        model = tf.keras.Model(inputs = inputs, outputs = outputs)
        #print(model.summary())
        ####################################
        # Training and evaluating the model
        ####################################
        # Using training data with or without augmentation
        sweep_train_data = train_aug_data if config.data_aug else train_data
        sweep_val_data = val_aug_data if config.data_aug else val_data # In any case, validation data is not augmented
        
        model.compile(optimizer=config.optimizer,
                      loss = tf.keras.losses.CategoricalCrossentropy(name = 'loss'),
                      metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'acc')])
        
        # Fitting the model and logging metrics (train_loss, train_acc, val_loss, val_acc) after every epoch
        model_hist = model.fit(sweep_train_data, epochs = config.num_epochs,
                               validation_data = sweep_val_data, 
                               callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5),
                                            wandb.keras.WandbCallback()])

In [9]:
pip install wandb

In [10]:
wandb.login()


wandb: Currently logged in as: go4rav (use `wandb login --relogin` to force relogin)


True

In [11]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="CS6910 Assignment2", entity="go4rav")

In [ ]:
#################################
# Setting up wandb sweeps
#################################
sweep_id = wandb.sweep(sweep_config, project = 'CS6910 Assignment2')
wandb.agent(sweep_id, function=CNN_train,count=40)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 4z474dtx
Sweep URL: https://wandb.ai/go4rav/CS6910%20Assignment2/sweeps/4z474dtx


wandb: Agent Starting Run: x0nri6rd with config:
wandb: 	activation: relu
wandb: 	batch_norm: False
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 512
wandb: 	dropout: 0.3
wandb: 	filter_org: 1
wandb: 	first_layer_filters: 32
wandb: 	kernel_size: 7
wandb: 	num_epochs: 20
wandb: 	optimizer: adam


Epoch 1/20
62/62 [==============================] - 90s 1s/step - loss: 2.3035 - acc: 0.1003 - val_loss: 2.3032 - val_acc: 0.1021 - _timestamp: 1649368044.0000 - _runtime: 95.0000
Epoch 2/20
62/62 [==============================] - 440s 7s/step - loss: 2.3027 - acc: 0.1053 - val_loss: 2.2920 - val_acc: 0.1042 - _timestamp: 1649368485.0000 - _runtime: 536.0000
Epoch 3/20
62/62 [==============================] - 559s 9s/step - loss: 2.2988 - acc: 0.1079 - val_loss: 2.3040 - val_acc: 0.0917 - _timestamp: 1649369044.0000 - _runtime: 1095.0000
Epoch 4/20
62/62 [==============================] - 541s 9s/step - loss: 2.3042 - acc: 0.0877 - val_loss: 2.3032 - val_acc: 0.0938 - _timestamp: 1649369585.0000 - _runtime: 1636.0000
Epoch 5/20
62/62 [==============================] - 99s 2s/step - loss: 2.3034 - acc: 0.0932 - val_loss: 2.3020 - val_acc: 0.1207 - _timestamp: 1649369684.0000 - _runtime: 1735.0000
Epoch 6/20
62/62 [==============================] - 82s 1s/step - loss: 2.3031 - acc: 0.10

wandb: Agent Starting Run: 06ylt48w with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 1024
wandb: 	dropout: 0.2
wandb: 	filter_org: 2
wandb: 	first_layer_filters: 64
wandb: 	kernel_size: 7
wandb: 	num_epochs: 10
wandb: 	optimizer: adam


...

Epoch 1/10
62/62 [==============================] - 28s 383ms/step - loss: 2.6400 - acc: 0.1305 - val_loss: 2.3084 - val_acc: 0.1021 - _timestamp: 1649376983.0000 - _runtime: 33.0000
Epoch 2/10
62/62 [==============================] - 25s 370ms/step - loss: 2.4658 - acc: 0.1648 - val_loss: 2.3229 - val_acc: 0.0938 - _timestamp: 1649377023.0000 - _runtime: 73.0000
Epoch 3/10
62/62 [==============================] - 25s 371ms/step - loss: 2.2961 - acc: 0.2056 - val_loss: 2.3514 - val_acc: 0.1042 - _timestamp: 1649377048.0000 - _runtime: 98.0000
Epoch 4/10
62/62 [==============================] - 25s 370ms/step - loss: 2.2748 - acc: 0.2061 - val_loss: 2.4032 - val_acc: 0.1125 - _timestamp: 1649377089.0000 - _runtime: 139.0000
Epoch 5/10
62/62 [==============================] - 25s 373ms/step - loss: 2.1886 - acc: 0.2278 - val_loss: 2.5071 - val_acc: 0.1000 - _timestamp: 1649377130.0000 - _runtime: 180.0000
Epoch 6/10
62/62 [==============================] - 25s 373ms/step - loss: 2.1162 -

wandb: Agent Starting Run: 0e9imlps with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: False
wandb: 	dense_size: 1024
wandb: 	dropout: 0
wandb: 	filter_org: 1
wandb: 	first_layer_filters: 64
wandb: 	kernel_size: 7
wandb: 	num_epochs: 20
wandb: 	optimizer: adam


Epoch 1/20
62/62 [==============================] - 35s 442ms/step - loss: 2.5116 - acc: 0.1628 - val_loss: 2.8296 - val_acc: 0.1063 - _timestamp: 1649377344.0000 - _runtime: 41.0000
Epoch 2/20
62/62 [==============================] - 29s 433ms/step - loss: 2.2040 - acc: 0.2308 - val_loss: 2.7256 - val_acc: 0.1292 - _timestamp: 1649377380.0000 - _runtime: 77.0000
Epoch 3/20
62/62 [==============================] - 29s 435ms/step - loss: 2.1343 - acc: 0.2555 - val_loss: 2.4488 - val_acc: 0.1375 - _timestamp: 1649377409.0000 - _runtime: 106.0000
Epoch 4/20
62/62 [==============================] - 29s 431ms/step - loss: 2.1068 - acc: 0.2631 - val_loss: 2.4667 - val_acc: 0.1688 - _timestamp: 1649377449.0000 - _runtime: 146.0000
Epoch 5/20
62/62 [==============================] - 29s 429ms/step - loss: 2.0461 - acc: 0.2863 - val_loss: 2.8416 - val_acc: 0.1667 - _timestamp: 1649377478.0000 - _runtime: 175.0000
Epoch 6/20
62/62 [==============================] - 28s 428ms/step - loss: 1.9829 

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: o1zg7y0n with config:
wandb: 	activation: relu
wandb: 	batch_norm: False
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 1024
wandb: 	dropout: 0
wandb: 	filter_org: 1
wandb: 	first_layer_filters: 32
wandb: 	kernel_size: 3
wandb: 	num_epochs: 20
wandb: 	optimizer: adam


Epoch 1/20
62/62 [==============================] - 85s 1s/step - loss: 2.3011 - acc: 0.1134 - val_loss: 2.3038 - val_acc: 0.1208 - _timestamp: 1649378120.0000 - _runtime: 91.0000
Epoch 2/20
62/62 [==============================] - 83s 1s/step - loss: 2.2861 - acc: 0.1341 - val_loss: 2.2627 - val_acc: 0.1530 - _timestamp: 1649378260.0000 - _runtime: 231.0000
Epoch 3/20
62/62 [==============================] - 83s 1s/step - loss: 2.2637 - acc: 0.1552 - val_loss: 2.2428 - val_acc: 0.1312 - _timestamp: 1649378343.0000 - _runtime: 314.0000
Epoch 4/20
62/62 [==============================] - 83s 1s/step - loss: 2.2231 - acc: 0.1779 - val_loss: 2.2163 - val_acc: 0.1729 - _timestamp: 1649378426.0000 - _runtime: 397.0000
Epoch 5/20
62/62 [==============================] - 82s 1s/step - loss: 2.2035 - acc: 0.1830 - val_loss: 2.1855 - val_acc: 0.2271 - _timestamp: 1649378508.0000 - _runtime: 479.0000
Epoch 6/20
62/62 [==============================] - 81s 1s/step - loss: 2.1693 - acc: 0.2041 - v